<a href="https://colab.research.google.com/github/gustavoborgesguimaraes/python-projects/blob/main/desafio_autoglass_nivelamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autor: Gustavo Borges Guimarães

# Desafio: Consumo de Dados para Previsão do Tempo dos Municípios do ES.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo dos municípios do ES. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/estados/32/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada município. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do ES, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do ES. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Vila Velha. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e zipar juntamente com os CSVs das duas tabelas e postar no grupo do Teams.



-----------------------------------------------
### Caso queira utilizar o colab do google para executar o script Spark descomente as linhas abaixo

In [2]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [4]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2023-01-24 17:54:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  3.49MB/s    in 4.8s    

2023-01-24 17:54:20 (2.76 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[],"uri":"/api/tunnels"}


---------------------------------------------------

In [5]:
# use o config [.config('spark.ui.port', '4050') \] caso queira ver o spark UI no colab, caso contrário pode retirar esta configuração
!pip install -q findspark
!pip install -q unidecode
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .config('spark.ui.port', '4050') \
      .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 7.6 MB/s eta 0:00:00


In [ ]:
# Buscar cidades do Espírito Santo
# TODO

# Criar data frame com as cidades
# TODO

# Criar view com as cidades
# TODO

In [ ]:
# Buscar previsão do tempo para as cidades
# TODO

# Criar data frame com as previsões
# TODO

# Criar view com as previsões
# TODO

In [ ]:
# Criar DF da Tabela 1
# TODO

In [ ]:
# Criar DF da Tabela 2
# TODO

In [ ]:
# Exportar CSVs
# TODO

In [ ]:
# Realizando a requisição HTTP com o método GET, utilizando a URL abaixo

import json
from pprint import pprint

requisicao = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados/32/municipios")
with open("/content/response-api-ibge.json") as file:
  data = json.load(file)

pprint(data)

In [29]:
# Realizando a requisição HTTP com o método GET, utilizando a URL abaixo

ResponseApi = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados/32/municipios")
ResponseJson = ResponseApi.json()

with open('FileRespJson.json', 'w') as json_file:
  json.dump(ResponseJson, json_file, indent=4)

In [38]:
# Realizando a requisição HTTP com o método GET, utilizando a URL abaixo
import pandas as pd

ResponseApi = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados/32/municipios")
ResponseJson = ResponseApi.json()

df = pd.read_json("/content/FileRespJson.json")
df.head()

,id,nome,microrregiao,regiao-imediata
0,3200102,Afonso Cláudio,"{'id': 32007, 'nome': 'Afonso Cláudio', 'mesor...","{'id': 320002, 'nome': 'Afonso Cláudio - Venda..."
1,3200136,Águia Branca,"{'id': 32002, 'nome': 'Nova Venécia', 'mesorre...","{'id': 320005, 'nome': 'Colatina', 'regiao-int..."
2,3200169,Água Doce do Norte,"{'id': 32001, 'nome': 'Barra de São Francisco'...","{'id': 320006, 'nome': 'Nova Venécia', 'regiao..."
3,3200201,Alegre,"{'id': 32011, 'nome': 'Alegre', 'mesorregiao':...","{'id': 320008, 'nome': 'Alegre', 'regiao-inter..."
4,3200300,Alfredo Chaves,"{'id': 32010, 'nome': 'Guarapari', 'mesorregia...","{'id': 320001, 'nome': 'Vitória', 'regiao-inte..."
